# Genetic Algorithm

- stochastic global optimization algorithm
- inspired by the biological theory of evolution by means of natural selection with a binary representation and simple operators based on genetic recombination and genetic mutations.
- one iteration of the algorithm is like an *evolutionary generation*.

In [55]:
from numpy.random import randint, rand

## Genetic Algorithm From Scratch

In [56]:
n_pop = 10 # population size
n_bits = 5 # number of bits a single candidate solution
n_iter = 3 # iteration step

# initialize the population (bitstring)
pop = [randint(0,2,n_bits).tolist() for _ in range(n_pop)] 
# tolist(): convert from np.array to list

In [57]:
def genetic_object(c):
	return randint(0,3)
scores = [genetic_object(c) for c in pop]
scores

[2, 2, 2, 2, 2, 1, 1, 0, 0, 1]

In [58]:
def tournament(pop, scores, k=3):
    """
    Get k parents randomly from population and choose the best of 3
    """
    idxs = randint(0, len(pop),k).tolist()
    selection_idx = idxs[0]
    for idx in idxs:
        if scores[idx] <scores[selection_idx]:
            selection_idx = idx
    return pop[selection_idx]

In [59]:
selected = [tournament(pop,scores) for _ in range(len(pop))]
selected

[[0, 0, 1, 1, 1],
 [0, 0, 1, 1, 1],
 [1, 1, 0, 1, 1],
 [0, 0, 1, 0, 1],
 [1, 0, 0, 1, 0],
 [1, 1, 0, 1, 1],
 [1, 0, 0, 1, 0],
 [1, 0, 0, 1, 0],
 [0, 1, 1, 0, 0],
 [0, 1, 1, 0, 0]]

## Create the next generation with `crossover`

In [60]:
def crossover(p1, p2, cross_rate = .95):
	"""
	Cross over the pair (p1, p2) with high probability (.95) 
	"""
	c1 = p1.copy()
	c2 = p2.copy()
	
	if rand() < cross_rate:
		cross_point = randint(1,len(p1)-2) # not the end of the string
		c1 = p1[cross_point:] + p2[:cross_point]
		c2= p2[cross_point:] + p1[:cross_point]
	return [c1, c2]
def mutation(bitstring, mut_rate = .05):
	"""
	Mutation the string with low probability (.05)
	"""
	for i in range(len(bitstring)):
		if rand() < mut_rate:
			bitstring[i] = 1- bitstring[i]
	return bitstring
print(pop[0])
print(mutation(pop[0]))

[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]


In [61]:
def next_gen(pop, r_cross, r_mut):
	selected = [tournament(pop, scores) for _ in range(n_pop)]
	# create the next generation
	children = list()
	for i in range(0, n_pop, 2):
		# get selected parents in pairs
		p1, p2 = selected[i], selected[i+1]
		# crossover and mutation
		for c in crossover(p1, p2, r_cross):
			# mutation
			mutation(c, r_mut)
			# store for next generation
			children.append(c)
	return children

def next_generation(pop, scores):
	selected = [tournament(pop, scores) for _ in range(n_pop)]
	pairs = [[selected[i], selected[i+1]] for i in range(0,n_pop,2)]
	new_gen = [crossover(p1,p2) for p1, p2 in pairs]
	new_gen = sum(new_gen,[])
	new_gen = [mutation(new_gen[i]) for i in range(n_pop)]
	return new_gen
# next_generation(pop), \
# next_gen(pop, .95,.05)

In [62]:
pop[0]

[1, 1, 1, 1, 1]

In [63]:
def genetic_algorithm(objective, n_pop, n_bits, n_iters, cross_rate, mut_rate):
	"""
	Tournament to select the best parents
	Crossover them, make offsprings
	Mutate their offsprings
	"""
	# Initialize a population
	pop = [randint(0,2,n_bits).tolist() for _ in range(n_pop)]
	best, best_eval = None, objective(pop[0])

	for iteration in range(n_iters):
		# Calculate scores
		scores = [objective(parent) for parent in pop]

		# Display the best generation
		for i in range(n_pop):
			if scores[i] < best_eval:
				best_eval, best = scores[i], pop[i]
				print(">%d: Best generation: f(%s) = %d" %(iteration, best, best_eval))

		# Select the best parent
		good_parents = [tournament(pop,scores) for _ in range(n_pop)]


		# Crossover parents and mutation children
		children = list()
		for i in range(0, n_pop, 2):
			c = crossover(good_parents[i], good_parents[i+1], cross_rate)
			children.append(mutation(c[0],mut_rate))
			children.append(mutation(c[1],mut_rate))

		# Replace old population with new generation
		pop = children
	return best, best_eval

In [64]:
# objective function
def onemax(x):
	return -sum(x)

In [65]:
n_pop = 100
n_bits = 20
n_iters = 10
mut_rate = 1.0/float(n_bits)
cross_rate = 1
best, score = genetic_algorithm(onemax, n_pop, n_bits, n_iters, cross_rate, mut_rate)
best, score

>0: Best generation: f([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]) = -12
>0: Best generation: f([1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1]) = -13
>0: Best generation: f([0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0]) = -14
>1: Best generation: f([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]) = -15
>2: Best generation: f([1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]) = -16
>2: Best generation: f([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]) = -17
>4: Best generation: f([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -19
>7: Best generation: f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20


([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -20)

## Optimize a continuous function

In [66]:
def objective(x):
	return x[0]**2.0 + x[1]**2

In [67]:
# bits per variable 
n_bits = 16

# Define range of 2 variables
bounds = [[-.5,.5],[-.5,.5]]

# Update mut_rate due to 2 variables
mut_rate = 1.0/float(n_bits * len(bounds))

n_pop = 10
# Initial population of random bitstring
pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]

In [68]:
# Decode from stringbits to float number in defined range
def decode(bounds, n_bits, bitstring):
	decoded = list()
	upper_bound = 2**(n_bits)
	for i, bound in enumerate(bounds):
		
		#divide into substring
		sub_bitstring = bitstring[i*n_bits:(i+1)*n_bits]
		
		# convert sub_bitstring to string of bits
		string_number = ''.join(str(bit) for bit in sub_bitstring)

		# convert string of bits into decimal
		number = int(string_number, base=2)

		# scale decimal number into desired range
		number = bound[0] + (bound[1]- bound[0]) * (number/upper_bound)
		decoded.append(number)
	return decoded
decode(bounds, n_bits,pop[2])

[0.4886627197265625, -0.174102783203125]

In [69]:
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	# enumerate generations
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [tournament(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

In [70]:
# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([-0.1483154296875, -0.249481201171875]) = 0.084238
>0, new best f([0.150146484375, 0.082855224609375]) = 0.029409
>1, new best f([0.14892578125, 0.082855224609375]) = 0.029044
>2, new best f([0.150146484375, 0.004730224609375]) = 0.022566
>5, new best f([0.021209716796875, 0.024261474609375]) = 0.001038
Done!
f([0.021209716796875, 0.024261474609375]) = 0.001038


## Refs
https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/